# CS838 Project Stage3

Yuying Chen
Maria Kamenetsky

In [ ]:
#!which -a python
#!echo $PATH

In [ ]:
#import modules
import csv
import py_entitymatching as em
import os
import sys
import re
import time
import shutil
import collections
import pandas as pd
from sklearn import tree
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import KFold
import random

## Read input tables

Since the original two movie dataset that we selected don't have too many overlaps, we changed one of them to another movie dataset. The first dataset is from kaggle.com. The second dataset is from movielens.org.

We added an id column for both datasets.

In [ ]:
A = em.read_csv_metadata('../../DataforProject/movie1_stage3.csv', key='id',encoding='mac_roman')
B = em.read_csv_metadata('../../DataforProject/movie2_stage3.csv', key='movieId',encoding='mac_roman')

In [ ]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A x B: ' + str(len(A)*len(B)))

In [ ]:
A.head() 

In [ ]:
B.head()

In [ ]:
em.get_key(A), em.get_key(B)

Since the tables are very large let's downsample:

In [ ]:
A1, B1 = em.down_sample(A,B, 200,1, show_progress=True)
len(A1), len(B1)

## Block Tables To Get Candidate Set

Block on attribute 'year' using an overlap blocker since the same movie must have the same year. The output attributes of table A is limited to id, title, year and genres to match the information provided by table B.

In [ ]:
# Create overlap blocker
ob = em.OverlapBlocker()
# Block tables using 'year' attribute 
#C = ob.block_tables(sample_A, sample_B, 'title_year', 'year', 
 #                   l_output_attrs=['id', 'movie_title', 'title_year', 'genres'], 
#                  r_output_attrs=['movieId', 'title', 'year', 'genres'],
   #                 overlap_size=1, show_progress=True
                    #)
C = ob.block_tables(A, B, 'year', 'year',
                   l_output_attrs=['id', 'title', 'year', 'genres'],
                   r_output_attrs=['movieId', 'title','year', 'genres'],
                   overlap_size=1, show_progress=True)
em.to_csv_metadata(C, './candidate_pairs.csv') #I really like this function

In [ ]:
len(C)

## Match tuple pairs in candidate set

### Import Pre-Determined Golden Data

We randomly sampled 500 tuple pairs for labeling. There are 201 matches and 299 mismatches. **gold.csv** is our gold dataset of perfect matches that we have determined prior to this. 

In [ ]:
#G = pd.read_csv("../../DataforProject/gold.csv", encoding='mac_roman')
#len(G)

In [ ]:
G = em.read_csv_metadata('../../DataforProject/gold.csv', 
                         key='_id',
                            ltable=A, rtable=B, 
                        fk_ltable='ltable_id', fk_rtable='rtable_movieId',
                         encoding='mac_roman')
len(G)

In [ ]:
G.head(10)

In [ ]:
#G = em.label_table(G, 'gold')

In [ ]:
We note here, that based on what we saw in the creation of the golden data, we establish a *correct* match as:
    - 

### Splitting Labeled Data into Development and Evaluation Set

We next want to split our data into a development set, where we will iteratively train and test various classifiers on the data. The evaluation set will be our final hold out set which we will test last and only once, so that we avoid overfitting. 

Let *I* be the development set and *J* be the evaluation set. We split the data into a 70-30 split for development-evaluation.

In [ ]:
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

**Package note**: the above split_train_test did not work if I imported G using pandas read_csv, even with encoding. I had to import it using
    ```em.read_csv_metadata()```, which is a little confusing to first-time user since it's a csv, but not necessarily metadata.
    Is there another function that would import metadata and do something with it separately? Or a way to interface with ```pd.read_csv()``` so that  you can read in a csv, clean it and work with it, and then reimport with ```em.read_csv_metadata()``` somehow?

### Selecting Best Learning-Based Matcher

We first go through and set our various machine-learning matchers that we will cycle through:

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In exploring the **py_entitymatching**, we first start with using the automatically generated features.

In [ ]:
#get the auto features
feature_table = em.get_features_for_matching(A,B)

feature_table.feature_name

**Package Note**: The above failed with error: `KeyError: "['feature_name' 'left_attribute' 'right_attribute' 'left_attr_tokenizer'\n 'right_attr_tokenizer' 'simfunction' 'function' 'function_source'\n 'is_auto_generated'] not in index"`


Also, the ```get_features_for_matching()``` doesn't work if A and B do not have any common features. Which totally makes sense, but the error message doesn't make this clear for sleepy people.

In [ ]:
#convert to feature vectors
H = em.extract_feature_vecs(I,
                           feature_table=feature_table,
                           attrs_after='label',
                           show_progress=True)

In [ ]:
H.head()

### Selecting best ML Matcher

In [ ]:
Now we want to select the best matcher. We use k-fold cross validation. We will use 10 folds here and use precision, recall, and F1 to select the best matcher.

